In [1]:
import os
import cv2
import time
import joblib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, accuracy_score, precision_score

tf.__version__

'2.4.1'

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# -------- TEST USER ----------- #

TEST_USER      = '001'

BASE_DIR       = '../'

# Google Drive
# BASE_DIR       = '/content/drive/MyDrive/Research/Hand Gesture/GitHub/'

IMG_DIR        = 'BW-Spatial-Path-Images/'
LOG_DIR        = 'Logs/'

USERS          = ['001', '002', '003', '004', '005', '006', '007']

# ------------------------------- Only Dynalic Gestures ------------------------------ #
GESTURES       = ['j', 'z', 'bad', 'deaf', 'fine', 'good', 'goodbye', 'hello', 'hungry',
                  'me', 'no', 'please', 'sorry', 'thankyou', 'yes', 'you']

PLANES         = ['XY', 'YZ', 'ZX']

BATCH_SIZE     = 32
IMG_LEN        = 160
IMG_SIZE       = (IMG_LEN, IMG_LEN)

# ------------- FOR THE GREATER GOOD :) ------------- #
DATASET_LEN    = 1120
TRAIN_LEN      = 960
TEST_LEN       = 160

EPOCHS         = 7
LEARNING_RATE  = 0.001
DECAY          = 0.0

CONFIG         = '_L_7_S_160x160_E_7'

XY_WEIGHTS     = np.array([0.91, 0.75, 0.61, 0.63, 0.51, 0.66, 0.81, 0.65, 0.65, 0.31,
                           0.66, 0.29, 0.34, 0.64, 0.64, 0.31])
YZ_WEIGHTS     = np.array([0.73, 0.71, 0.70, 0.79, 0.76, 0.38, 0.80, 0.61, 0.58, 0.73,
                           0.49, 0.26, 0.26, 0.52, 0.59, 0.54])
ZX_WEIGHTS     = np.array([0.33, 0.66, 0.51, 0.54, 0.37, 0.51, 0.71, 0.30, 0.75, 0.41,
                           0.40, 0.27, 0.24, 0.61, 0.36, 0.49])

In [4]:
def load_data(plane):
    X_train = np.zeros((TRAIN_LEN, IMG_LEN, IMG_LEN, 3))
    X_test = np.zeros((TEST_LEN, IMG_LEN, IMG_LEN, 3))
    y_train = np.zeros((TRAIN_LEN, 1))
    y_test = np.zeros((TEST_LEN, 1))
    
    train_count = 0
    test_count = 0
        
    for gesture in GESTURES:
        print('loading data for ' + gesture + ' gesture on the ' + plane + ' plane ... ', end='')
        path = os.path.join(BASE_DIR, IMG_DIR, plane, gesture)
        for filename in os.listdir(path):
            img = cv2.imread(os.path.join(path, filename))
            resized = cv2.resize(img, IMG_SIZE)
            if filename[1:4] != TEST_USER:
                X_train[train_count, :] = resized
                y_train[train_count, 0] = GESTURES.index(gesture)
                train_count = train_count + 1
            else:
                X_test[test_count, :] = resized
                y_test[test_count, 0] = GESTURES.index(gesture)
                test_count = test_count + 1
                
        print('√')
        
    return X_train, X_test, y_train, y_test

def load_and_save_data(plane):
    X = np.zeros((DATASET_LEN, IMG_LEN, IMG_LEN, 3))
    y = np.zeros((DATASET_LEN, 1))
    
    train_count = 0
    test_count  = TRAIN_LEN
        
    for gesture in GESTURES:
        print('loading data for ' + gesture + ' gesture on the ' + plane + ' plane ... ', end='')
        path = os.path.join(BASE_DIR, IMG_DIR, plane, gesture)
        for filename in os.listdir(path):
            img = cv2.imread(os.path.join(path, filename))
            resized = cv2.resize(img, IMG_SIZE)
            if filename[1:4] != TEST_USER:
                X[train_count, :] = resized
                y[train_count, 0] = GESTURES.index(gesture)
                train_count = train_count + 1
            else:
                X[test_count, :] = resized
                y[test_count, 0] = GESTURES.index(gesture)
                test_count = test_count + 1
                
        print('√')

    joblib.dump(X, BASE_DIR + 'X_BW_' + plane + str(IMG_SIZE) + '.joblib')
    joblib.dump(y, BASE_DIR + 'Y_BW_' + plane + str(IMG_SIZE) + '.joblib')

def load_data_from_joblib(plane):
    print('Loading data for ' + plane + ' plane ... ', end='')
    X = joblib.load(BASE_DIR + 'X_BW_' + plane + str(IMG_SIZE) + '.joblib')
    y = joblib.load(BASE_DIR + 'Y_BW_' + plane + str(IMG_SIZE) + '.joblib')
    test_user = int(TEST_USER)
    X_train = X[:TRAIN_LEN, :, :, :]
    y_train = y[:TRAIN_LEN, :]
    X_test = X[TRAIN_LEN:, :, :, :]
    y_test = y[TRAIN_LEN:, :]

    print('√')

    return X_train, X_test, y_train, y_test

In [5]:
X_train_xy, X_test_xy, y_train_xy, y_test_xy = load_data('XY')
X_train_yz, X_test_yz, y_train_yz, y_test_yz = load_data('YZ')
X_train_zx, X_test_zx, y_train_zx, y_test_zx = load_data('ZX')

# Save to Google  Drive
# load_and_save_data('XY')
# load_and_save_data('YZ')
# load_and_save_data('ZX')

# Load from Google Drive
# X_train_xy, X_test_xy, y_train_xy, y_test_xy = load_data_from_joblib('XY')
# X_train_yz, X_test_yz, y_train_yz, y_test_yz = load_data_from_joblib('YZ')
# X_train_zx, X_test_zx, y_train_zx, y_test_zx = load_data_from_joblib('ZX')

loading data for j gesture on the XY plane ... √
loading data for z gesture on the XY plane ... √
loading data for bad gesture on the XY plane ... √
loading data for deaf gesture on the XY plane ... √
loading data for fine gesture on the XY plane ... √
loading data for good gesture on the XY plane ... √
loading data for goodbye gesture on the XY plane ... √
loading data for hello gesture on the XY plane ... √
loading data for hungry gesture on the XY plane ... √
loading data for me gesture on the XY plane ... √
loading data for no gesture on the XY plane ... √
loading data for please gesture on the XY plane ... √
loading data for sorry gesture on the XY plane ... √
loading data for thankyou gesture on the XY plane ... √
loading data for yes gesture on the XY plane ... √
loading data for you gesture on the XY plane ... √
loading data for j gesture on the YZ plane ... √
loading data for z gesture on the YZ plane ... √
loading data for bad gesture on the YZ plane ... √
loading data for de

In [6]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [7]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model.trainable = False

In [8]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(len(GESTURES))

In [9]:
def get_model():
    inputs = tf.keras.Input(shape=IMG_SHAPE)
#     x = data_augmentation(inputs)
    x = preprocess_input(inputs)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE, decay=DECAY),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [10]:
model_xy = get_model()
X_train_xy, y_train_xy = shuffle(X_train_xy, y_train_xy)
history_xy = model_xy.fit(X_train_xy, y_train_xy, validation_data=(X_test_xy, y_test_xy), epochs=EPOCHS)

Epoch 1/7
30/30 [==============================] - 15s 446ms/step - loss: 2.9415 - accuracy: 0.1347 - val_loss: 1.8386 - val_accuracy: 0.4187
Epoch 2/7
30/30 [==============================] - 12s 407ms/step - loss: 1.7064 - accuracy: 0.4817 - val_loss: 1.4330 - val_accuracy: 0.5000
Epoch 3/7
30/30 [==============================] - 12s 408ms/step - loss: 1.2644 - accuracy: 0.6377 - val_loss: 1.1871 - val_accuracy: 0.6062
Epoch 4/7
30/30 [==============================] - 12s 400ms/step - loss: 0.9809 - accuracy: 0.6952 - val_loss: 1.0656 - val_accuracy: 0.6687
Epoch 5/7
30/30 [==============================] - 12s 406ms/step - loss: 0.8981 - accuracy: 0.7421 - val_loss: 1.0314 - val_accuracy: 0.6625
Epoch 6/7
30/30 [==============================] - 12s 406ms/step - loss: 0.7117 - accuracy: 0.8064 - val_loss: 0.9842 - val_accuracy: 0.6812
Epoch 7/7
30/30 [==============================] - 12s 404ms/step - loss: 0.7166 - accuracy: 0.7704 - val_loss: 0.9872 - val_accuracy: 0.6750


In [11]:
# prob_xy = tf.keras.Sequential([model_xy, tf.keras.layers.Softmax()])
# y_pred_xy = prob_xy.predict(X_test_xy)
y_pred_xy = model_xy.predict(X_test_xy)
y_pred = np.argmax(y_pred_xy, axis=1)
print(classification_report(y_test_xy.ravel(), y_pred, zero_division=0))
prc_xy = precision_score(y_test_xy.ravel(), y_pred, zero_division=0, average=None)
tf.keras.backend.clear_session()

              precision    recall  f1-score   support

         0.0       1.00      0.90      0.95        10
         1.0       1.00      0.20      0.33        10
         2.0       0.88      0.70      0.78        10
         3.0       1.00      0.90      0.95        10
         4.0       1.00      0.90      0.95        10
         5.0       0.00      0.00      0.00        10
         6.0       1.00      0.70      0.82        10
         7.0       0.42      1.00      0.59        10
         8.0       0.73      0.80      0.76        10
         9.0       0.33      0.80      0.47        10
        10.0       0.90      0.90      0.90        10
        11.0       0.67      0.60      0.63        10
        12.0       0.75      0.30      0.43        10
        13.0       0.58      0.70      0.64        10
        14.0       0.60      0.90      0.72        10
        15.0       0.83      0.50      0.62        10

    accuracy                           0.68       160
   macro avg       0.73   

In [12]:
model_yz = get_model()
X_train_yz, y_train_yz = shuffle(X_train_yz, y_train_yz)
history_yz = model_yz.fit(X_train_yz, y_train_yz, validation_data=(X_test_yz, y_test_yz), epochs=EPOCHS)

Epoch 1/7
30/30 [==============================] - 15s 429ms/step - loss: 3.5235 - accuracy: 0.1381 - val_loss: 2.0106 - val_accuracy: 0.2812
Epoch 2/7
30/30 [==============================] - 12s 405ms/step - loss: 1.8710 - accuracy: 0.4029 - val_loss: 1.3693 - val_accuracy: 0.5750
Epoch 3/7
30/30 [==============================] - 12s 402ms/step - loss: 1.2346 - accuracy: 0.5973 - val_loss: 1.1736 - val_accuracy: 0.6438
Epoch 4/7
30/30 [==============================] - 12s 401ms/step - loss: 0.9772 - accuracy: 0.6532 - val_loss: 1.0740 - val_accuracy: 0.6875
Epoch 5/7
30/30 [==============================] - 12s 400ms/step - loss: 0.7661 - accuracy: 0.7698 - val_loss: 1.0207 - val_accuracy: 0.6438
Epoch 6/7
30/30 [==============================] - 12s 407ms/step - loss: 0.6420 - accuracy: 0.7967 - val_loss: 0.9661 - val_accuracy: 0.6438
Epoch 7/7
30/30 [==============================] - 12s 403ms/step - loss: 0.6094 - accuracy: 0.8232 - val_loss: 1.0144 - val_accuracy: 0.6438


In [13]:
# prob_yz = tf.keras.Sequential([model_yz, tf.keras.layers.Softmax()])
# y_pred_yz = prob_yz.predict(X_test_yz)
y_pred_yz = model_yz.predict(X_test_yz)
y_pred = np.argmax(y_pred_yz, axis=1)
print(classification_report(y_test_yz.ravel(), y_pred, zero_division=0))
prc_yz = precision_score(y_test_yz.ravel(), y_pred, zero_division=0, average=None)
tf.keras.backend.clear_session()

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        10
         1.0       1.00      0.60      0.75        10
         2.0       0.60      0.30      0.40        10
         3.0       0.67      1.00      0.80        10
         4.0       0.77      1.00      0.87        10
         5.0       0.50      0.20      0.29        10
         6.0       0.50      0.50      0.50        10
         7.0       0.69      0.90      0.78        10
         8.0       0.69      0.90      0.78        10
         9.0       1.00      1.00      1.00        10
        10.0       0.77      1.00      0.87        10
        11.0       0.42      1.00      0.59        10
        12.0       0.00      0.00      0.00        10
        13.0       0.71      1.00      0.83        10
        14.0       0.67      0.20      0.31        10
        15.0       0.50      0.70      0.58        10

    accuracy                           0.64       160
   macro avg       0.59   

In [14]:
model_zx = get_model()
X_train_zx, y_train_zx = shuffle(X_train_zx, y_train_zx)
history_zx = model_zx.fit(X_train_zx, y_train_zx, validation_data=(X_test_zx, y_test_zx), epochs=EPOCHS)

Epoch 1/7
30/30 [==============================] - 15s 434ms/step - loss: 3.5243 - accuracy: 0.1947 - val_loss: 2.5679 - val_accuracy: 0.3313
Epoch 2/7
30/30 [==============================] - 12s 403ms/step - loss: 2.2328 - accuracy: 0.3394 - val_loss: 2.0181 - val_accuracy: 0.3688
Epoch 3/7
30/30 [==============================] - 12s 405ms/step - loss: 1.6689 - accuracy: 0.4817 - val_loss: 1.6968 - val_accuracy: 0.4688
Epoch 4/7
30/30 [==============================] - 12s 404ms/step - loss: 1.3536 - accuracy: 0.5804 - val_loss: 1.4971 - val_accuracy: 0.5063
Epoch 5/7
30/30 [==============================] - 12s 404ms/step - loss: 1.1209 - accuracy: 0.6452 - val_loss: 1.3891 - val_accuracy: 0.5437
Epoch 6/7
30/30 [==============================] - 12s 413ms/step - loss: 1.0074 - accuracy: 0.6828 - val_loss: 1.3800 - val_accuracy: 0.5312
Epoch 7/7
30/30 [==============================] - 12s 414ms/step - loss: 0.9219 - accuracy: 0.7178 - val_loss: 1.2834 - val_accuracy: 0.5437


In [15]:
# prob_zx = tf.keras.Sequential([model_zx, tf.keras.layers.Softmax()])
# y_pred_zx = prob_zx.predict(X_test_zx)
y_pred_zx = model_zx.predict(X_test_zx)
y_pred = np.argmax(y_pred_zx, axis=1)
print(classification_report(y_test_zx.ravel(), y_pred, zero_division=0))
prc_zx = precision_score(y_test_zx.ravel(), y_pred, zero_division=0, average=None)
tf.keras.backend.clear_session()

              precision    recall  f1-score   support

         0.0       0.67      1.00      0.80        10
         1.0       0.80      0.80      0.80        10
         2.0       0.46      0.60      0.52        10
         3.0       0.91      1.00      0.95        10
         4.0       0.00      0.00      0.00        10
         5.0       0.77      1.00      0.87        10
         6.0       0.62      1.00      0.77        10
         7.0       0.40      0.20      0.27        10
         8.0       1.00      0.90      0.95        10
         9.0       0.50      0.30      0.37        10
        10.0       0.38      0.80      0.52        10
        11.0       0.13      0.20      0.16        10
        12.0       0.50      0.10      0.17        10
        13.0       0.56      0.50      0.53        10
        14.0       0.00      0.00      0.00        10
        15.0       1.00      0.30      0.46        10

    accuracy                           0.54       160
   macro avg       0.54   

In [16]:
y_total = y_pred_xy + y_pred_yz + y_pred_zx
y_pred = np.argmax(y_total, axis=1)
report = classification_report(y_test_xy.ravel(), y_pred, zero_division=0)
print(report)

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        10
         1.0       1.00      0.90      0.95        10
         2.0       1.00      0.80      0.89        10
         3.0       1.00      1.00      1.00        10
         4.0       1.00      1.00      1.00        10
         5.0       1.00      1.00      1.00        10
         6.0       1.00      1.00      1.00        10
         7.0       0.91      1.00      0.95        10
         8.0       1.00      1.00      1.00        10
         9.0       1.00      1.00      1.00        10
        10.0       1.00      1.00      1.00        10
        11.0       0.53      1.00      0.69        10
        12.0       1.00      0.10      0.18        10
        13.0       1.00      1.00      1.00        10
        14.0       0.80      0.80      0.80        10
        15.0       0.83      1.00      0.91        10

    accuracy                           0.91       160
   macro avg       0.94   

In [17]:
# config = '\n\nTEST_USER ' + TEST_USER + ' T: ' + str(int(time.time())) + '\n'
# underline = '=====================================\n'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'logs_sptl_bw' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(underline)
# f.write(report)
# f.close()

In [18]:
# config = TEST_USER + ' :'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'prc_sptl_bw_xy' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(np.array2string(prc_xy, precision=2, max_line_width=100) + '\n')
# f.close()

In [19]:
# config = TEST_USER + ' :'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'prc_sptl_bw_yz' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(np.array2string(prc_yz, precision=2, max_line_width=100) + '\n')
# f.close()

In [20]:
# config = TEST_USER + ' :'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'prc_sptl_bw_zx' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(np.array2string(prc_zx, precision=2, max_line_width=100) + '\n')
# f.close()